In [ ]:
import requests
from bs4 import BeautifulSoup

In [ ]:
!pip install flask-ngrok

In [ ]:
%tensorflow_version 1.x
!pip install -q gpt-2-simple
import gpt_2_simple as gpt2

In [ ]:
from flask import Flask,request
from flask_ngrok import run_with_ngrok
app = Flask(__name__)
run_with_ngrok(app)

from keras import backend as K
K.clear_session()
gpt2.copy_checkpoint_from_gdrive(run_name='run2')
sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess, run_name='run2')

import tensorflow as tf
graph = tf.get_default_graph()
global old_text
old_text=""
@app.route("/")
def root():
    html="<b><center><h2>Predict next word - Using GPT2<br>Fine Tuned on book - Inglorious Empire by Dr. Shashi Tharoor<br>Amrita Univ</h2></center></b><br><br>"
    html=html+'<br>\
    <form method="POST" action="/submit"><br>\
    Text: <textarea style="width: 500px;height: 100px;" name="txt" id="txt"></textarea><br><br>\
    <input type="submit" value="predict"/> <br><br>\
    Enter threshold(eg: 0.7): <input  name="threshold" id="threshold" value="0.6"></input><br><br>\
    <br><br><br>---------------- \
    </form>'
    
    return html

@app.route("/submit",methods=['POST'])
def submit():

    global old_text
    txt=request.form['txt']
    
    threshold=float(request.form['threshold'])
    with graph.as_default():
      pred=gpt2.generate(sess, run_name='run2',length=5,temperature=threshold,prefix=txt,nsamples=5,batch_size=5,return_as_list=True)
    pred=[p.replace('\n',' ').rstrip() for p in pred]
    html="<b><center><h2>Predict next word - Using GPT2<br>Fine Tuned on book - Inglorious Empire by Dr. Shashi Tharoor<br>Amrita Univ</h2></center></b><br><br>"
    html=html+'<br>\
    <form method="POST" action="/submit"><br>\
    Text: <textarea style="width: 500px;height: 100px;" name="txt" id="txt" >'+txt+pred[0][len(txt):]+'</textarea><br><br>\
    <input type="submit" value="Continue predict"/><br><br> \
    Top Predictions: <textarea style="width: 500px;height: 100px;" name="txt1" id="txt1" >'+"\n".join(pred)+'</textarea><br><br>\
    Enter threshold(eg: 0.7): <input  name="threshold" id="threshold" value="0.8"></input><br><br>\
    <br><br><br>---------------- \
    </form>'
    old_text=txt
    return html

if __name__ == "__main__":
    app.run()